In [ ]:
import os
import psycopg
from datetime import date
from googleapiclient.discovery import build
from dotenv import load_dotenv
from tqdm import tqdm

load_dotenv()

# Collecter et enregistrer les métriques des chaînes

- On va collecter et enregistrer en même temps les métriques de chaque chaîne pertinente (en évitant de passer par un JSON).


In [ ]:
youtube = build('youtube', 'v3', developerKey=os.getenv("YOUTUBE_API_KEY1"))

In [ ]:
def getMetrics(id_chaine):
    request = youtube.channels().list(
        part='statistics',
        id=id_chaine
    )
    response = request.execute()
    
    if not response['items']:
        return None
    
    stats = response['items'][0]['statistics']
    date_releve_chaine = date.today().isoformat()
    nombre_vues_total = int(stats.get('viewCount', 0))
    nombre_abonnes_total = int(stats.get('subscriberCount', 0))
    nombre_videos_total = int(stats.get('videoCount', 0))
    
    return {
        'date_releve_chaine': date_releve_chaine,
        'nombre_vues_total': nombre_vues_total,
        'nombre_abonnes_total': nombre_abonnes_total,
        'nombre_videos_total': nombre_videos_total
    }

In [ ]:
getMetrics("UCVQeGg4Fdrrr8vDXa7yjOYg")

In [ ]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)
cur = conn.cursor()

# Récupérer les chaînes pertinentes
cur.execute("SELECT id_chaine FROM chaines WHERE pertinente = TRUE")
chaines = cur.fetchall()


print(chaines)
print(len(chaines))

In [ ]:
# Pour chaque chaîne, récupérer et insérer les métriques
for (id_chaine,) in chaines:
    metriques = getMetrics(id_chaine)
    if metriques:
        cur.execute("""
            INSERT INTO chaines_metriques (
                id_chaine, date_releve_chaine,
                nombre_vues_total, nombre_abonnes_total, nombre_videos_total
            )
            VALUES (%s, %s, %s, %s, %s)
            ON CONFLICT (id_chaine, date_releve_chaine) DO NOTHING
        """, (
            id_chaine,
            metriques['date_releve_chaine'],
            metriques['nombre_vues_total'],
            metriques['nombre_abonnes_total'],
            metriques['nombre_videos_total']
        ))

conn.commit()
cur.close()
conn.close()

# Calcul de la couverture

$$
\text{Couverture} = \frac{\text{Nombre de vidéos pertinentes collectées}}{\text{Nombre total de vidéos (le plus récent) de la chaîne pertinente}}
$$


In [ ]:
def get_couverture(id_chaine,conn):
    
    cur = conn.cursor()

    # Récupérer le nombre de vidéos collectées
    cur.execute("""
        SELECT COUNT(*) FROM videos
        WHERE id_chaine = %s
    """, (id_chaine,))
    nb_collectees = cur.fetchone()[0]

    # Récupérer le nombre total de vidéos le plus récent
    cur.execute("""
        SELECT nombre_videos_total
        FROM chaines_metriques
        WHERE id_chaine = %s
        ORDER BY date_releve_chaine DESC
        LIMIT 1
    """, (id_chaine,))
    row = cur.fetchone()


    if not row:
        return None  # Pas de métrique pour cette chaîne

    nb_total = row[0]

    if nb_total == 0:
        return 0.0  # Évite division par zéro

    couverture = nb_collectees / nb_total
    return round(couverture, 3)


In [ ]:
conn = psycopg.connect(
        dbname="youtubestay",
        user="postgres",
        password=os.getenv("POSTGRE_PASSWORD"),
        host="localhost",
        port="5432"
    )

couverture = get_couverture("UCxBJustR1tuXVy7tLivER2g",conn)
print("Couverture :", couverture)

conn.close()


In [ ]:
def get_couverture_moyenne():
    conn = psycopg.connect(
        dbname="youtubestay",
        user="postgres",
        password=os.getenv("POSTGRE_PASSWORD"),
        host="localhost",
        port="5432"
    )
    cur = conn.cursor()

    cur.execute("""
        SELECT id_chaine FROM chaines WHERE pertinente = TRUE
    """)
    chaines = cur.fetchall()

    total = 0.0
    count = 0

    for (id_chaine,) in tqdm(chaines):
        couverture = get_couverture(id_chaine,conn)
        if couverture is not None:
            total += couverture
            count += 1

    conn.close()

    if count == 0:
        return 0.0
    return round(total / count, 3)


In [ ]:
get_couverture_moyenne()

# Enrichissement des vidéos

In [ ]:
from googleapiclient.discovery import build
import re
import scrapetube

youtube = build("youtube", "v3", developerKey=os.getenv("YOUTUBE_API_Nou2008"))

In [ ]:

def getVideosIdsYoutubeAPI(channel_id):

    channel_response = youtube.channels().list(
        part='contentDetails',
        id=channel_id
    ).execute()

    uploads_playlist_id = channel_response['items'][0]['contentDetails']['relatedPlaylists']['uploads']

    video_ids = []
    next_page_token = None

    while True:
        playlist_response = youtube.playlistItems().list(
            part='snippet',
            playlistId=uploads_playlist_id,
            maxResults=50,
            pageToken=next_page_token
        ).execute()

        for item in playlist_response['items']:
            video_id = item['snippet']['resourceId']['videoId']
            video_ids.append(video_id)

        next_page_token = playlist_response.get('nextPageToken')
        if not next_page_token:
            break

    return video_ids

def toSeconds(iso_duration):
    # Exemple : "PT50M11S" → 3011 secondes
    match = re.match(r'PT(?:(\d+)H)?(?:(\d+)M)?(?:(\d+)S)?', iso_duration)
    if not match:
        return None
    hours = int(match.group(1)) if match.group(1) else 0
    minutes = int(match.group(2)) if match.group(2) else 0
    seconds = int(match.group(3)) if match.group(3) else 0
    return hours * 3600 + minutes * 60 + seconds

def getvideos_details_bunch(video_ids):
    try:
        
        request = youtube.videos().list(
            part='snippet,contentDetails',
            id=','.join(video_ids[:50])  # Max 50 IDs per request
        )
        response = request.execute()
    except Exception as e:
        print(f"probleme with request  {e}")

    videos_metadata = []

    for item in response.get('items', []):
        video_metadata = {
            'id_video': item['id'],
            'id_chaine': item['snippet']['channelId'],
            'titre_video': item['snippet']['title'],
            'description': item['snippet'].get('description', ''),
            'date_publication': item['snippet']['publishedAt'],
            'duree': item['contentDetails']['duration'],
            'miniature': '',
            'tags': '',
            'langue': 'fr'
        }

        resolution_order = ["maxres", "standard", "high", "medium", "default"]
        for res in resolution_order:
            if res in item['snippet'].get('thumbnails', {}):
                video_metadata['miniature'] = item['snippet']['thumbnails'][res]['url']
                break

        if 'tags' in item['snippet']:
            video_metadata['tags'] = item['snippet']['tags']

        videos_metadata.append(video_metadata)

    return videos_metadata


In [ ]:
ids = getVideosIdsYoutubeAPI("UCbXqpYCVTjdz2dx1nRx6W_A")
len(ids)

In [8]:
conn = psycopg.connect(
    dbname="youtubestay",
    user="postgres",
    password=os.getenv("POSTGRE_PASSWORD"),
    host="localhost",
    port="5432"
)

cur = conn.cursor()

In [ ]:
getvideos_details_bunch(ids)

In [ ]:
cur.execute("""
    SELECT id_chaine FROM chaines WHERE pertinente = TRUE
""")

chaines_pertinentes = cur.fetchall()

for (id_chaine,) in tqdm(chaines_pertinentes): 
    videos_ids = getVideosIdsYoutubeAPI(id_chaine)
    
    for i in range(0, len(videos_ids), 50):
        batch_ids = videos_ids[i:i+50]  # Paquet de 50 IDs max
        videos = getvideos_details_bunch(batch_ids)
      
        data_to_insert = []

        for video in videos:
            id_video = video["id_video"]
            id_chaine = video["id_chaine"]
            titre = video["titre_video"]
            description = video["description"]
            date_publication = video["date_publication"][:10]  # 'YYYY-MM-DD'
            duree = toSeconds(video["duree"])
            miniature = video["miniature"]
            langue = video["langue"]
            transcription = None
            tags = video["tags"] if isinstance(video["tags"], list) else None
            requetes = None
            categorie_video = None

            data_to_insert.append((
                id_video, titre, description, date_publication, categorie_video,
                duree, miniature, langue, transcription, tags, requetes, id_chaine
            ))

        if data_to_insert:
            cur.executemany("""
                INSERT INTO videos (
                    id_video, titre, description, date_publication, categorie_video,
                    duree, miniature, langue, transcription, tags, requetes, id_chaine
                ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                ON CONFLICT (id_video) DO NOTHING
            """, data_to_insert)


100%|██████████| 280/280 [08:05<00:00,  1.73s/it]


In [10]:
conn.commit()
cur.close()
conn.close()

# Collecter et enregistrer les métriques des vidéos